# Modelo linguístico

Modelos Seq2Seq mapeiam sequências de tokens para outras sequências de tokens. Em geral, eles têm um par encoder-decoder. O encoder mapeia uma sequência inteira para 

In [1]:
from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf
import keras
import numpy as np

In [28]:
# Refazer estrutura do dataset: uma frase por arquivo, arquivos em um diretorio
import pandas as pd
import os
import re
from tqdm import tqdm

#df = pd.read_csv('datasets/IMDB Dataset.csv')
#DATASET_DIR = './frases_imdb/'
# DATASET_DIR = './datasets/frases/'
DATASET_DIR = './datasets/frases_classificacao/train/positive/'


In [29]:

# n_texto = 0
# for i in tqdm(range(len(df))):
#     texto = df.iloc[i]['review']
#     #print(texto)
#     texto = texto.replace('<br />', '.')
#     #texto = re.sub(r'[\n \t][\n \t][\n \t]+', '.', texto)
#     texto = re.split(r'[.!?\n\t-]', texto)
#     #print(texto)
#     for t in texto:

#         t = t.lstrip().rstrip()
#         if len(t) > 5:
#             #print(t)
#             fname = 'review_' + str(n_texto) + '.txt'
#             with open(DATASET_DIR + fname, 'w') as f:
#                 f.write(t)
#             n_texto += 1

# print("Criei textos:", n_texto)


In [30]:
# Ler um dataset e fazer batches
from tensorflow.keras.utils import text_dataset_from_directory

dataset = text_dataset_from_directory(
    DATASET_DIR,
    labels=None,
    label_mode='int',
    class_names=None,
    batch_size=2048,
    max_length=None,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    follow_links=False
)

Found 19937 files belonging to 1 classes.


In [31]:
from keras.layers import Input, TextVectorization
from keras.models import Model

vocab_size = 5000
vectorize_layer = TextVectorization(max_tokens=vocab_size, output_sequence_length=10)
vectorize_layer.adapt(dataset)

In [32]:
print(vectorize_layer.get_vocabulary())

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'br', 'as', 'with', 'for', 'was', 'but', 'film', 'movie', 'his', 'on', 'you', 'are', 'he', 'not', 'one', 'its', 'have', 'be', 'by', 'an', 'at', 'all', 'who', 'from', 'her', 'they', 'has', 'so', 'like', 'about', 'very', 'out', 'or', 'if', 'more', 'what', 'when', 'good', 'some', 'just', 'there', 'she', 'my', 'great', 'story', 'see', 'their', 'which', 'time', 'up', 'well', 'can', 'also', 'really', 'would', 'will', 'had', 'we', 'me', 'were', 'only', 'even', 'him', 'most', 'other', 'than', 'much', 'first', 'into', 'no', 'films', 'love', 'people', 'how', 'best', 'get', 'been', 'because', 'way', 'many', 'do', 'after', 'life', 'them', 'characters', 'think', 'movies', 'made', 'two', 'too', 'seen', 'then', 'dont', 'watch', 'character', 'being', 'little', 'still', 'show', 'make', 'never', 'where', 'could', 'does', 'man', 'any', 'know', 'did', 'while', 'years', 'ever', 'these', 'over', 'end', 'such', 'those', 'scen

In [33]:
# vocab = vectorize_layer.get_vocabulary()
# vocab = vocab[0:2] + ['[SOS]'] + vocab[2:-1]
# vectorize_layer.set_vocabulary(vocab)
# print(vectorize_layer.get_vocabulary())

In [34]:
def separar_ultimo_token(x):
    x_ = vectorize_layer(x)
    x_ = x_[:,:-1]
    y_ = x_[:,-1:]
    return x_, y_

Qual a saida do classificar? Sai a probabilidade de cada palavra do seu vocabulário
Qual a entrada dele? uma frase 

Ignora palavras nao existentes

In [35]:
def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = LSTM(latent_dim, kernel_initializer='glorot_uniform')(x)#kernel initializar diz sobre os pesos dos kernels #mask zero é para mascarar os zeros e nao rodar sem motivo
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(10, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 15)             75000     
                                                                 
 lstm_2 (LSTM)               (None, 15)                1860      
                                                                 
 dense_2 (Dense)             (None, 5000)              80000     
                                                                 
 softmax_2 (Softmax)         (None, 5000)              0         
                                                                 
Total params: 156,860
Trainable params: 156,860
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = predictor.fit(dataset.map(separar_ultimo_token), epochs=20, verbose=1)


Epoch 1/20
10/10 [==============================] - 5s 263ms/step - loss: 6.8101 - accuracy: 0.0298
Epoch 2/20
10/10 [==============================] - 3s 225ms/step - loss: 4.8560 - accuracy: 0.1050
Epoch 3/20
10/10 [==============================] - 3s 201ms/step - loss: 3.5529 - accuracy: 0.3239
Epoch 4/20
10/10 [==============================] - 3s 229ms/step - loss: 2.6307 - accuracy: 0.5081
Epoch 5/20
10/10 [==============================] - 3s 215ms/step - loss: 2.0463 - accuracy: 0.6014
Epoch 6/20
10/10 [==============================] - 2s 152ms/step - loss: 1.6611 - accuracy: 0.6559
Epoch 7/20
10/10 [==============================] - 3s 230ms/step - loss: 1.4081 - accuracy: 0.6851
Epoch 8/20
10/10 [==============================] - 2s 156ms/step - loss: 1.2264 - accuracy: 0.7048
Epoch 9/20
10/10 [==============================] - 4s 366ms/step - loss: 1.0869 - accuracy: 0.7166
Epoch 10/20
10/10 [==============================] - 3s 200ms/step - loss: 0.9779 - accuracy: 0.7301

Retorna a palavra mais provavel depois

In [37]:
pred = predictor.predict(vectorize_layer(["this movie is"])[:,:-1])
print(pred)
idx = tf.argmax(pred, axis=1)[0]
print(idx)
word = vectorize_layer.get_vocabulary()[idx]
print(word)

1/1 [==============================] - 4s 4s/step
[[3.7838228e-07 3.5728638e-07 9.0773879e-03 ... 1.6849135e-03
  3.7065590e-07 7.6613578e-06]]
tf.Tensor(1744, shape=(), dtype=int64)
mixed


In [16]:
def predizer(entrada, numero_de_predicoes, modelo, vectorize_layer):
    frase = entrada
    for n in range(numero_de_predicoes):
        pred = modelo.predict(vectorize_layer([frase])[:,:-1])
        idx = tf.argmax(pred, axis=1)[0]
        word = vectorize_layer.get_vocabulary()[idx]
        frase = frase + " " + word
        print(word)
    return frase

predizer("this movie is", 2, predictor, vectorize_layer)

1/1 [==============================] - 0s 41ms/step

1/1 [==============================] - 0s 19ms/step



'this movie is  '

In [106]:
def predizer2(entrada, numero_de_predicoes, modelo, vectorize_layer):
    frase = entrada
    contexto = frase # Contexto deslizante
    for n in range(numero_de_predicoes):
        pred = modelo.predict(vectorize_layer([contexto])[:,:-1])

        # Nao repetir palavras
        tentando = True
        while tentando:

            # Selectionar de k-best
            candidatos = tf.math.top_k(pred, k=10).indices[0,:]
            idx = np.random.choice(candidatos.numpy())
#            idx = tf.argmax(pred, axis=1)[0]
            word = vectorize_layer.get_vocabulary()[idx]
            if word in frase.split():
                pred[0][idx] = 0
            else:
                tentando = False
                
        frase = frase + " " + word
        contexto = contexto + " " + word
        #print(frase)
        contexto = ' '.join(frase.split()[1:])
        print(word)
    return frase

predizer2("i like how", 10, predictor, vectorize_layer)

1/1 [==============================] - 0s 81ms/step
but
1/1 [==============================] - 0s 41ms/step
or
1/1 [==============================] - 0s 55ms/step
off
1/1 [==============================] - 0s 52ms/step
chance
1/1 [==============================] - 0s 100ms/step
visual
1/1 [==============================] - 0s 41ms/step
100
1/1 [==============================] - 0s 44ms/step
hero
1/1 [==============================] - 0s 64ms/step
cannot
1/1 [==============================] - 0s 46ms/step
footage
1/1 [==============================] - 0s 83ms/step
came


'i like how but or off chance visual 100 hero cannot footage came'